In [576]:
#Imports
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [577]:

df = pd.read_csv('../data/hotel_bookings.csv')
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,city
0,Resort Hotel - Chandigarh,0,342,2024,July,30,27,0,0,2,...,NaN,NaN,0,Transient,0.0,0,0,Check-Out,16:40.9,Chandigarh
1,Resort Hotel - Mumbai,0,737,2024,April,17,28,0,0,2,...,NaN,NaN,0,Transient,0.0,0,0,Check-Out,56:21.5,Mumbai
2,Resort Hotel - Delhi,0,7,2024,September,37,10,0,1,1,...,NaN,NaN,0,Transient,75.0,0,0,Check-Out,46:25.7,Delhi
3,Resort Hotel - Kolkata,0,13,2024,August,33,14,0,1,1,...,304.0,NaN,0,Transient,75.0,0,0,Check-Out,07:10.1,Kolkata
4,Resort Hotel - Lucknow,0,14,2024,September,37,14,0,2,2,...,240.0,NaN,0,Transient,98.0,0,1,Check-Out,27:32.5,Lucknow


In [578]:
df.shape

(119390, 33)

In [579]:
missing = df.isnull().sum()
missing[missing > 0]

children         4
country        488
agent        16340
company     112593
dtype: int64

In [580]:
# Fill missing values
df['children'] = df['children'].fillna(0)  # Assume no children if missing
df['country'] = df['country'].fillna('Unknown')
df['agent'] = df['agent'].fillna(0)  # 0 means no agent
df['company'] = df['company'].fillna(0)  # 0 means no company

# Verify
df.isnull().sum().sum()

np.int64(0)

In [581]:
df.dtypes


hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [582]:
df['children'] = df['children'].astype(int)
df['agent'] = df['agent'].astype(int)
df['company'] = df['company'].astype(int)

df[['children', 'agent', 'company']].dtypes


children    int64
agent       int64
company     int64
dtype: object

In [583]:
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}
df['arrival_date_month'] = df['arrival_date_month'].map(month_map)

df['arrival_date_month'].head()


0    7
1    4
2    9
3    8
4    9
Name: arrival_date_month, dtype: int64

In [584]:
df.columns.tolist()

['hotel',
 'is_canceled',
 'lead_time',
 'arrival_date_year',
 'arrival_date_month',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'reserved_room_type',
 'assigned_room_type',
 'booking_changes',
 'deposit_type',
 'agent',
 'company',
 'days_in_waiting_list',
 'customer_type',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'reservation_status',
 'reservation_status_date',
 'city']

In [585]:
df.duplicated().sum()

np.int64(0)

In [586]:
df = df.drop_duplicates()

In [587]:
df.shape


(119390, 33)

In [588]:
#remove invalid rows with 0 people

df = df[~((df['adults'] == 0) & (df['children'] == 0) & (df['babies'] == 0))]

df.shape


(119210, 33)

In [589]:
df['lead_time'].describe()

count    119210.000000
mean        104.109227
std         106.875450
min           0.000000
25%          18.000000
50%          69.000000
75%         161.000000
max         737.000000
Name: lead_time, dtype: float64

In [590]:
# Remove negative ADR
df = df[df['adr'] >= 0]

# Cap extreme high values at 99th percentile
cap = df['adr'].quantile(0.99)
df['adr'] = df['adr'].clip(upper=cap)

In [591]:
df['adr'].describe()

count    119209.000000
mean        101.599515
std          46.790008
min           0.000000
25%          69.500000
50%          94.950000
75%         126.000000
max         252.000000
Name: adr, dtype: float64

In [592]:
# Cap other numeric outliers
num_cols = ['lead_time', 'stays_in_week_nights', 'stays_in_weekend_nights']
for col in num_cols:
    cap = df[col].quantile(0.99)
    df[col] = df[col].clip(upper=cap)
df[num_cols].describe()

,lead_time,stays_in_week_nights,stays_in_weekend_nights
count,119209.000000,119209.000000,119209.000000
mean,103.505314,2.478982,0.919620
std,104.610755,1.758419,0.956348
min,0.000000,0.000000,0.000000
25%,18.000000,1.000000,0.000000
50%,69.000000,2.000000,1.000000
75%,161.000000,3.000000,2.000000
max,444.000000,10.000000,4.000000


In [593]:
cat_cols = df.select_dtypes(include=['object']).columns
cat_cols

Index(['hotel', 'meal', 'country', 'market_segment', 'distribution_channel',
       'reserved_room_type', 'assigned_room_type', 'deposit_type',
       'customer_type', 'reservation_status', 'reservation_status_date',
       'city'],
      dtype='object')

In [594]:
for col in cat_cols:
    print(f"{col}: {df[col].nunique()} unique values")

hotel: 30 unique values
meal: 5 unique values
country: 178 unique values
market_segment: 8 unique values
distribution_channel: 5 unique values
reserved_room_type: 9 unique values
assigned_room_type: 11 unique values
deposit_type: 3 unique values
customer_type: 4 unique values
reservation_status: 3 unique values
reservation_status_date: 34543 unique values
city: 15 unique values


In [595]:
le = LabelEncoder()
df['country_encoded'] = le.fit_transform(df['country'])
df = df.drop(columns=['country'])

In [596]:
df['country_encoded'].head()

0    135
1    135
2     59
3     59
4     59
Name: country_encoded, dtype: int64

In [597]:
cat_to_onehot = ['hotel', 'meal', 'market_segment', 'distribution_channel', 
                 'reserved_room_type', 'assigned_room_type', 
                 'deposit_type', 'customer_type', 'city']

df = pd.get_dummies(df, columns=cat_to_onehot, drop_first=True)

In [598]:
df.shape

(119209, 105)

In [599]:
#handle rare categories in 'country_encoded'
country_counts = df['country_encoded'].value_counts()
rare_countries = country_counts[country_counts < 30].index  # threshold can be tuned

df['country_encoded'] = df['country_encoded'].replace(rare_countries, -1)

In [600]:
df['total_stay'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']
df['total_people'] = df['adults'] + df['children'] + df['babies']

In [601]:
df[['stays_in_weekend_nights', 'stays_in_week_nights', 'total_stay','total_people']].head()

,stays_in_weekend_nights,stays_in_week_nights,total_stay,total_people
0,0,0,0,2
1,0,0,0,2
2,0,1,1,1
3,0,1,1,1
4,0,2,2,2


In [602]:
# Avoid division by zero
df['adr_per_person'] = df['adr'] / df['total_people'].replace(0, 1)

# Family indicator
df['is_family'] = ((df['children'] + df['babies']) > 0).astype(int)

df[['total_stay', 'total_people', 'adr_per_person', 'is_family']].head()

,total_stay,total_people,adr_per_person,is_family
0,0,2,0.0,0
1,0,2,0.0,0
2,1,1,75.0,0
3,1,1,75.0,0
4,2,2,49.0,0


In [604]:
# Drop useless or leaking columns
cols_to_drop = [
    'company',                 # ID-like, mostly useless
    'agent',                   # ID-like
    'reservation_status',      # Leaks cancellation info
    'reservation_status_date'  # Already extracted features
]

df = df.drop(columns=cols_to_drop, errors='ignore')


In [605]:
# Check class balance
df['is_canceled'].value_counts(normalize=True)


is_canceled
0    0.629231
1    0.370769
Name: proportion, dtype: float64

In [606]:
df.head()

,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,city_Jaipur,city_Kochi,city_Kolkata,city_Lucknow,city_Mumbai,city_Pune,total_stay,total_people,adr_per_person,is_family
0,0,342,2024,7,30,27,0,0,2,0,...,False,False,False,False,False,False,0,2,0.0,0
1,0,444,2024,4,17,28,0,0,2,0,...,False,False,False,False,True,False,0,2,0.0,0
2,0,7,2024,9,37,10,0,1,1,0,...,False,False,False,False,False,False,1,1,75.0,0
3,0,13,2024,8,33,14,0,1,1,0,...,False,False,True,False,False,False,1,1,75.0,0
4,0,14,2024,9,37,14,0,2,2,0,...,False,False,False,True,False,False,2,2,49.0,0


In [607]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols.remove('is_canceled')

scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


In [608]:
df.to_csv('../data/hotel_bookings_cleaned.csv', index=False)

In [609]:

os.path.getsize('../data/hotel_bookings_cleaned.csv')

110696382